### Merging R parameter comparison tables

In [36]:
import os
import numpy as np
from matchms.importing import load_from_mgf
import pandas as pd
from itertools import compress

In [37]:
path_data = "/Users/krv114/Desktop/MS/Similarity comparison/conference_2023"
file_mgf_reference = os.path.join(path_data, "ex_spectra_stds_NEG_scans_50.mgf")
reference_spectra = list(load_from_mgf(file_mgf_reference))

In [38]:
file_mgf_query = os.path.join(path_data, "pseudo_query_neg_v1.mgf")
query_spectra = list(load_from_mgf(file_mgf_query))

Set up melt df with compound name_collision energy for the reference spectra and scan id_collision energy for the query

In [39]:
res = []

dir_path = path_data + '/parameter_test_NEG'

# Iterate directory
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)

In [40]:
ref_spec_feature_ids = [s.metadata['feature_id'] for s in reference_spectra]
query_spec_scan_ids = [s.metadata['scanindex'] for s in query_spectra]

In [41]:
out_df = pd.DataFrame()
for i, res_file in enumerate(res):
    new_colnames = []
    test_df = pd.read_csv(os.path.join(dir_path, res_file), sep=',')
    for feature_id in list(test_df.columns)[:-1]:
        ref_spec = reference_spectra[ref_spec_feature_ids.index(feature_id)]
        new_colnames.append(ref_spec.metadata['compound_name'] + '_' + ref_spec.metadata['collision_energy'])
    test_df.columns = new_colnames + ['id_query']
    new_rownames = []
    for scan_index in list(test_df['id_query'].values):
        query_spec = query_spectra[query_spec_scan_ids.index(str(scan_index))]
        new_rownames.append(query_spec.metadata['scanindex'] + '_' + query_spec.metadata['collision_energy'])
    test_df['id_query'] = new_rownames
    #melt_df = pd.melt(test_df, id_vars=test_df.columns[-1], var_name='id_query', value_name=res_file)
    melt_df = pd.melt(test_df, id_vars='id_query', value_name=res_file, var_name='ref_id')
    if i == 0:
        out_df = melt_df
    else:
        out_df[res_file] = list(melt_df[res_file].values)

In [42]:
out_df.to_csv(os.path.join(path_data, 'parameter_tests_NEG.csv'), index=False)

In [43]:
out_df = pd.read_csv(os.path.join(path_data, 'parameter_tests_NEG.csv'))

In [44]:
out_df

,id_query,ref_id,scores_nspecangle_m_0_n_1_tolerance_0.01.csv,scores_nspecangle_m_0.5_n_1_tolerance_0.005.csv,scores_cosine_m_0_n_1_tolerance_0.01.csv,scores_cosine_m_1_n_0.5_tolerance_0.005.csv,scores_cosine_m_1_n_2_tolerance_0.005.csv,scores_navdis_m_2_n_2_tolerance_0.005.csv,scores_navdis_m_2_n_1_tolerance_0.005.csv,scores_cosine_m_1_n_1_tolerance_0.005.csv,...,scores_neuclidean_m_0_n_2_tolerance_0.005.csv,scores_navdis_m_0_n_2_tolerance_0.01.csv,scores_nspecangle_m_0_n_0.5_tolerance_0.01.csv,scores_cosine_m_0_n_0.5_tolerance_0.005.csv,scores_neuclidean_m_2_n_1_tolerance_0.01.csv,scores_neuclidean_m_0.5_n_2_tolerance_0.01.csv,scores_navdis_m_0_n_0.5_tolerance_0.01.csv,scores_neuclidean_m_1_n_2_tolerance_0.01.csv,scores_navdis_m_0.5_n_0.5_tolerance_0.005.csv,scores_neuclidean_m_0_n_1_tolerance_0.005.csv
0,1_10,11-hydroxyeicosatetraenoic acid_10,0.0000,0.00000,0.000000,0.00000,0.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2_20,11-hydroxyeicosatetraenoic acid_10,0.0000,0.00000,0.000000,0.00000,0.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,3_30,11-hydroxyeicosatetraenoic acid_10,0.0000,0.00000,0.000000,0.00000,0.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,4_40,11-hydroxyeicosatetraenoic acid_10,0.0000,0.00000,0.000000,0.00000,0.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,5_10,11-hydroxyeicosatetraenoic acid_10,0.0000,0.00000,0.000000,0.00000,0.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562347,1610_20,Ursodeoxycholate_60,0.0012,0.00075,0.001886,0.00115,0.000052,0.999137,0.999536,0.000573,...,0.999929,0.997615,0.002257,0.003545,0.999997,0.999955,0.999329,0.999973,0.999467,0.999969
1562348,1611_10,Ursodeoxycholate_60,0.0000,0.00000,0.000000,0.00000,0.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1562349,1612_10,Ursodeoxycholate_60,0.0000,0.00000,0.000000,0.00000,0.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1562350,1613_10,Ursodeoxycholate_60,0.0000,0.00000,0.000000,0.00000,0.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [45]:
keep_row = []
for a, b in zip(out_df['id_query'].values, out_df['ref_id'].values):
    if a[-2:] == b[-2:]:
        keep_row.append(True)
    else:
        keep_row.append(False)
out_df = out_df[keep_row].reset_index(drop=True)

In [46]:
cols_to_keep = [False, False]
for col in list(out_df.columns)[2:]:
    cols_to_keep.append(all(out_df[col].values == 0) or all(out_df[col].values == 1))
cols_to_keep = [not elem for elem in cols_to_keep]

In [47]:
out_df_filtered = out_df[list(compress(list(out_df.columns), cols_to_keep))]

In [48]:
out_df_filtered.to_csv(os.path.join(path_data, 'parameter_tests_NEG_filtered.csv'), index=False)

In [49]:
out_df_filtered

,id_query,ref_id,scores_nspecangle_m_0_n_1_tolerance_0.01.csv,scores_nspecangle_m_0.5_n_1_tolerance_0.005.csv,scores_cosine_m_0_n_1_tolerance_0.01.csv,scores_cosine_m_1_n_0.5_tolerance_0.005.csv,scores_cosine_m_1_n_2_tolerance_0.005.csv,scores_navdis_m_2_n_2_tolerance_0.005.csv,scores_navdis_m_2_n_1_tolerance_0.005.csv,scores_cosine_m_1_n_1_tolerance_0.005.csv,...,scores_neuclidean_m_0_n_2_tolerance_0.005.csv,scores_navdis_m_0_n_2_tolerance_0.01.csv,scores_nspecangle_m_0_n_0.5_tolerance_0.01.csv,scores_cosine_m_0_n_0.5_tolerance_0.005.csv,scores_neuclidean_m_2_n_1_tolerance_0.01.csv,scores_neuclidean_m_0.5_n_2_tolerance_0.01.csv,scores_navdis_m_0_n_0.5_tolerance_0.01.csv,scores_neuclidean_m_1_n_2_tolerance_0.01.csv,scores_navdis_m_0.5_n_0.5_tolerance_0.005.csv,scores_neuclidean_m_0_n_1_tolerance_0.005.csv
0,1_10,11-hydroxyeicosatetraenoic acid_10,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,5_10,11-hydroxyeicosatetraenoic acid_10,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,6_10,11-hydroxyeicosatetraenoic acid_10,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,7_10,11-hydroxyeicosatetraenoic acid_10,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,9_10,11-hydroxyeicosatetraenoic acid_10,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315325,1580_60,Ursodeoxycholate_60,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
315326,1593_60,Ursodeoxycholate_60,0.000013,0.000021,0.000021,0.000316,4.548406e-08,0.490987,0.803757,0.000030,...,0.161453,0.607961,0.000236,0.000371,0.545594,0.125673,0.946817,0.103505,0.939116,0.698160
315327,1594_60,Ursodeoxycholate_60,0.001291,0.001526,0.002028,0.004271,8.681557e-05,0.031361,0.402661,0.002378,...,0.000209,0.048615,0.002822,0.004433,0.031499,0.000157,0.856625,0.000128,0.838722,0.056943
315328,1595_60,Ursodeoxycholate_60,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
